# Generate log file

In [24]:
%matplotlib inline
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import re

import pandas as pd

## List all logs files

In [25]:
import glob
files = glob.glob('../results/logs/**/events.out.tfevents.*', recursive=True)
#files = glob.glob('../results/logs_2019_03_09 _RBM_RBMCF_100_250_500_1000/**/events.out.tfevents.*', recursive=True)

len(files), files[0:3]

(5,
 ['../results/logs/kfold=2/kfold-intern=0/batch_size=10/class=RBMCF/visible_size=702/hidden_size=1000/regularization=NoRegularization-0.0/learning_rate=Adam-0.05-0.9-0.999-1e-08/sampling_method=CD-1/momentum=0/1557084657.908597/events.out.tfevents.1557084664.paulo-notebook',
  '../results/logs/kfold=4/kfold-intern=0/batch_size=10/class=RBMCF/visible_size=702/hidden_size=1000/regularization=NoRegularization-0.0/learning_rate=Adam-0.05-0.9-0.999-1e-08/sampling_method=CD-1/momentum=0/1557085696.6471844/events.out.tfevents.1557085702.paulo-notebook',
  '../results/logs/kfold=1/kfold-intern=0/batch_size=10/class=RBMCF/visible_size=702/hidden_size=1000/regularization=NoRegularization-0.0/learning_rate=Adam-0.05-0.9-0.999-1e-08/sampling_method=CD-1/momentum=0/1557084139.0567567/events.out.tfevents.1557084145.paulo-notebook'])

## Read a file

In [26]:
file = files[0]

def extract_information(file):
    ea = EventAccumulator(file)
    ea.Reload()
    
    scalars = ea.Tags()['scalars']
    first_scalar = ea.Scalars(scalars[0])
    
    dataframes = pd.DataFrame(first_scalar)[['wall_time', 'step']]
    
    for df_name in ea.Tags()['scalars']:
        df = pd.DataFrame(ea.Scalars(df_name))
        series = df['value'].rename(df_name)
        dataframes = pd.concat([dataframes, series], axis=1)
    
    del ea

    return dataframes

dfs = extract_information(file)
print(dfs.shape[0], 'evaluates')
print(dfs.shape[1], 'data columns')

dfs.head()

51 evaluates
38 data columns


,wall_time,step,measure/reconstruction/hamming,details/measure/top-1/evaluate-0/train,details/measure/top-1/evaluate-0/validation,details/measure/top-1/evaluate-1/train,details/measure/top-1/evaluate-1/validation,details/measure/top-1/evaluate-2/train,details/measure/top-1/evaluate-2/validation,details/measure/top-1/evaluate-3/train,...,details/measure/top-5/evaluate-5/validation,measure/evaluate/top-5/train,measure/evaluate/top-5/validation,measure/evaluate/Free/mean_free_energy_train,measure/evaluate/Free/mean_free_energy_validation,measure/evaluate/Free/ratio_mean_free_energy,measure/evaluate/Free/mean_free_energy_gap,measure/evaluate/Free/mean_free_energy_noisy,measure/evaluate/reconstruction/RMSE_train,measure/evaluate/reconstruction/RMSE_validation
0,1.557085e+09,1,3.073753,0.300072,0.358382,0.096168,0.092486,0.126537,0.086705,0.157628,...,0.647399,0.429501,0.432081,-57.850212,-57.786289,0.998895,14.069477,-43.716812,0.087927,0.087420
1,1.557085e+09,2,3.026753,0.300072,0.358382,0.096168,0.092486,0.126537,0.086705,0.084599,...,0.661850,0.442637,0.447013,-33.165699,-32.981026,0.994432,14.103245,-18.877781,0.087813,0.087348
2,1.557085e+09,3,3.053507,0.300072,0.358382,0.096168,0.092486,0.126537,0.086705,0.084599,...,0.710983,0.450952,0.459538,-20.848213,-20.562231,0.986283,13.821772,-6.740460,0.087736,0.087544
3,1.557085e+09,4,3.025307,0.300072,0.358382,0.096168,0.092486,0.126537,0.086705,0.157628,...,0.710983,0.456014,0.461946,-13.798235,-13.493020,0.977880,14.126879,0.633859,0.087627,0.087456
4,1.557085e+09,5,2.985539,0.300072,0.358382,0.096168,0.092486,0.140998,0.098266,0.157628,...,0.696532,0.450349,0.450867,-9.329809,-9.063185,0.971422,14.653622,5.590436,0.087460,0.087077


In [27]:
def extract_columns(file):
    regex = r'(.+?)=(.+?)\/'
    test_str = file.replace('../results/logs/', '')
    matches = re.finditer(regex, test_str, re.MULTILINE)

    columns = {match.groups()[0]: match.groups()[1] for match in matches}
    columns['path'] = test_str
    return columns

def populate_columns(file, dataframe):
    columns = extract_columns(file)
    
    for k, v in columns.items():
        dataframe[k] = v

    return dataframe

In [28]:
def generate_logs_files(files):
    data = []

    for index, file in enumerate(files):
        print(f'{index+1} of {len(files)}')
        information = extract_information(file)
        information = populate_columns(file, information)
        
        data.append(information)
        
    return pd.concat(data)

data = generate_logs_files(files)

1 of 5
2 of 5
3 of 5
4 of 5
5 of 5


In [29]:
data.index.name = 'evaluate'
data.head(5)

,wall_time,step,measure/reconstruction/hamming,details/measure/top-1/evaluate-0/train,details/measure/top-1/evaluate-0/validation,details/measure/top-1/evaluate-1/train,details/measure/top-1/evaluate-1/validation,details/measure/top-1/evaluate-2/train,details/measure/top-1/evaluate-2/validation,details/measure/top-1/evaluate-3/train,...,kfold-intern,batch_size,class,visible_size,hidden_size,regularization,learning_rate,sampling_method,momentum,path
evaluate,,,,,,,,,,,,,,,,,,,,,
0,1.557085e+09,1,3.073753,0.300072,0.358382,0.096168,0.092486,0.126537,0.086705,0.157628,...,0,10,RBMCF,702,1000,NoRegularization-0.0,Adam-0.05-0.9-0.999-1e-08,CD-1,0,kfold=2/kfold-intern=0/batch_size=10/class=RBM...
1,1.557085e+09,2,3.026753,0.300072,0.358382,0.096168,0.092486,0.126537,0.086705,0.084599,...,0,10,RBMCF,702,1000,NoRegularization-0.0,Adam-0.05-0.9-0.999-1e-08,CD-1,0,kfold=2/kfold-intern=0/batch_size=10/class=RBM...
2,1.557085e+09,3,3.053507,0.300072,0.358382,0.096168,0.092486,0.126537,0.086705,0.084599,...,0,10,RBMCF,702,1000,NoRegularization-0.0,Adam-0.05-0.9-0.999-1e-08,CD-1,0,kfold=2/kfold-intern=0/batch_size=10/class=RBM...
3,1.557085e+09,4,3.025307,0.300072,0.358382,0.096168,0.092486,0.126537,0.086705,0.157628,...,0,10,RBMCF,702,1000,NoRegularization-0.0,Adam-0.05-0.9-0.999-1e-08,CD-1,0,kfold=2/kfold-intern=0/batch_size=10/class=RBM...
4,1.557085e+09,5,2.985539,0.300072,0.358382,0.096168,0.092486,0.140998,0.098266,0.157628,...,0,10,RBMCF,702,1000,NoRegularization-0.0,Adam-0.05-0.9-0.999-1e-08,CD-1,0,kfold=2/kfold-intern=0/batch_size=10/class=RBM...


In [30]:
data.to_csv('extracted_log-minibatch10-2.csv')